# Onderzoeksvragen en eerste verkenning: FIFA 19

<b> Redouan el Hidraoui <br>
Maurits Arissen <br>
Eric Houdijk <br>
V2A </b>

Lever één Jupyter Notebook in met de volgende onderdelen:
* een eerste verkenning en analyse van je dataset (stappen 1 tot en met 4 uit het data science proces);
* drie concept-onderzoeksvragen;
* een externe dataset (als bijlage in csv-formaat).

## Data collection

*Data collection is al voor je gedaan, aangezien je een dataset krijgt toegewezen, 
waarbij de ruwe data al eerder is verzameld door derden.
Je moet nog wel op zoek naar een externe dataset om je data mee uit te breiden. 
Deze data is echter ook al eerder voor je verzameld: je hoeft zelf geen dataverzameling te doen door bijvoorbeeld interviews te houden of sensoren uit te lezen.*
Na deze stap is er sprake van <b>ruwe data</b>.

In [ ]:
# imports
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import holoviews as hv

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.mixture import GaussianMixture


hv.extension('bokeh')

Onze gekozen CSV-file dataset inladen: <br>
- fifa19.csv

De dataset bevat: <br>
1. Alle spelers uit FIFA 19.
2. Speelpositie en fysieke eigenschappen.
3. Persoonlijke spelersinformatie (bijv. nationaliteit, club, leeftijd, salaris etc.).

In [ ]:
fifa_source = pd.read_csv('source/fifa19.csv', index_col=0)

De grootte van onze dataset:
- 8.71 MB
- Bevat 18207 rijen (dus 18206 spelers, want de eerste rij bevat alle kolommen)

In [ ]:
rows_dataset = fifa_source.shape[0] - 1
rows_dataset

Een aantal rijen als voorbeeld voor hoe de dataset eruit ziet:

In [ ]:
fifa_source.head(10)

## Data processing

*Na het verzamelen van de data, moet je dit waarschijnlijk eerst bewerken, 
want het komt niet altijd in het formaat binnen dat je voor ogen had. 
Deze stap data processing is ook al grotendeels voor je gedaan: je krijgt de data immers in nette tabelvorm aangeleverd (bijvoorbeeld in CSV-files). 
Het is wel goed mogelijk dat je de data moet omvormen om meerdere datasets te kunnen combineren. 
Na deze stap heb je <b>geordende data</b>.*

De column <b>'Wage'</b>, de column <b>'Value'</b> en column <b>'Release Clause'</b> bevat characters zoals €, M en K. Om later gebruik te maken van deze data,
willen wij graag van deze characters af.

In [ ]:
fifa_source['Wage']

In [ ]:
fifa_source['Release Clause']

In [ ]:
fifa_source['Value']

Nu het verwijderen van de ongewenste characters <b>(Release Clause)</b>:

In [ ]:
fifa_source['Clean Release Clause'] = fifa_source['Release Clause']
fifa_source['Clean Release Clause'].replace(regex=True,inplace=True,to_replace=r'€',value=r'')

fifa_source['Clean Release Clause'] = fifa_source['Clean Release Clause'].replace(r'[KM]+$', '', regex=True).astype(float) * \
             fifa_source['Clean Release Clause'].str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int)

De <b>Release Clause</b> van de spelers na het opschonen:

In [ ]:
fifa_source['Clean Release Clause']

Nu het verwijderen van de ongewenste charachters <b>(Wage)</b>:

In [ ]:
fifa_source['Clean Wage'] = fifa_source['Wage']
fifa_source['Clean Wage'].replace(regex=True,inplace=True,to_replace=r'€',value=r'')

fifa_source['Clean Wage'] = fifa_source['Clean Wage'].replace(r'[KM]+$', '', regex=True).astype(float) * \
             fifa_source['Clean Wage'].str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int)

De <b>Wage</b> van de spelers na het opschonen:

In [ ]:
fifa_source['Clean Wage']

Nu het verwijderen van de ongewenste charachters <b>(Value)</b>:

In [ ]:
fifa_source['Clean Value'] = fifa_source['Value']
fifa_source['Clean Value'].replace(regex=True,inplace=True,to_replace=r'€',value=r'')

fifa_source['Clean Value'] = fifa_source['Clean Value'].replace(r'[KM]+$', '', regex=True).astype(float) * \
             fifa_source['Clean Value'].str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int)

De <b>Value</b> van de spelers na het opschonen:

In [ ]:
fifa_source['Clean Value']

## Data cleaning

*Je hebt nu een geordende dataset, maar deze is niet noodzakelijk al helemaal schoon. 
In de stap data cleaning ga je op zoek naar dubbele en missende waarden, outliers en andere onvolkomenheden en los je deze op. 
Na deze stap heb je <b>schone data</b>.*

Op dit moment bevat onze gekozen dataset <b>88 kolommen</b>:

In [ ]:
print('Aantal kolommen in fifa19.csv: {}'.format(len(fifa_source.columns)))
fifa_source.columns

Wij als team zijn tot de conclusie gekomen dat de volgende kolommen overbodig, en dus niet nodig zijn:
1. <b>Photo</b>, omdat: deze kolom een referentie bevat naar een afbeelding van de desbetreffende speler. Dit is nog relevant voor ons onderzoek nog hebben wij de beschikking tot deze foto's.
2. <b>Flag</b>, omdat: deze kolom een referentie bevat naar een afbeelding van de vlag die bij de nationaliteit hoort van de desbetreffende speler. Dit is nog relevant voor ons onderzoek, de 'Nationality' kolom is bruikbaarder, nog hebben wij beschikking tot deze afbeeldingen.
3. <b>Club logo</b>, omdat: bevat url's naar afbeeldingen
4. <b>Special</b>, omdat: nummers zonder duidelijke betekenis
5. <b>Body Type</b>, omdat: niet al te nuttig, erg generiek, bijna alle spelers delen dezelfde 3 bodytypes
6. <b>International Reputation</b>, omdat: weinig zeggend nummer
7. <b>Skill Moves</b>, omdat: weinig zeggend nummer
8. <b>Real Face</b>, omdat: Yes / No kolom, geen nuttige data
9. <b>Jersey Number</b>, omdat: weinig zeggend nummer, heeft geen specifiek betekenis

In [ ]:
fifa_source.drop(['Photo', 'Flag', 'Club Logo', 'Special', 'Body Type', 'International Reputation', 'Skill Moves', 'Real Face', 'Jersey Number'], axis=1, inplace=True)

De resterende kolommen na het verwijderen van de overbodige kolommen:

In [ ]:
print('Aantal kolommen in fifa19.csv na het verwijderen van overbodige kolommen: {}'.format(len(fifa_source.columns)))
fifa_source.columns

De column <b>'Overall'</b> bevat een getal (rating) van een speler. dit geeft weer hoe goed een speler
is t.o.v. de andere spelers binnen de dataset. Wij vinden dan ook dat deze column <b>niet</b> NaN mag zijn,
het heeft een belangrijke betekenis.
* Alle spelers die hier <b>NaN</b> hebben staan, worden uit onze dataset gehaald.

In [ ]:
aantal_nan_overall = pd.isnull(fifa_source['Overall']).sum().sum()
print('Aantal NaN-waardes column \'Overall\': {}'.format(aantal_nan_overall))

Onze dataset bevat geen spelers met NaN waardes in  column <b>'Overall'</b>.

De resterende columns met NaN worden door ons gevult met het getal 0. 
Dit doen wij om later (mochten die ontstaan door dit) foutmeldingen te voorkomen.

In [ ]:
aantal_nan = pd.isnull(fifa_source).sum().sum()
print('Huidige aantal NaN-waardes: {}'.format(aantal_nan))

In [ ]:
fifa_source.fillna(0, inplace=True)

In [ ]:
aantal_nan = pd.isnull(fifa_source).sum().sum()
print('Aantal NaN-waardes na het replacen met getal 0: {}'.format(aantal_nan))

Aantal duplicate spelers binnen onze fifa19.csv dataset:
* <b>0 spelers</b>

Dit checken wij aan de hand van het unieke ID die iedere speler toegewezen heeft gekregen.

In [ ]:
duplicatePlayers = fifa_source[fifa_source.duplicated(['ID'])]
print("Duplicate players gebaseerd op het unieke ID:", duplicatePlayers.count().sum())

## Data exploration & analysis

*Voordat je kan beginnen met het beantwoorden van de onderzoeksvragen, 
moet je de dataset eerst doorgronden en analyseren. 
Je moet begrijpen waar je data uit bestaat en hoe je met je data de onderzoeksvragen oplost. 
Gebruik <b>statistische analyses en visualisaties</b> om inzicht te krijgen in de dataset.* 

<b>Statische analyse column 'Overall'</b>:

In [ ]:
np.round(fifa_source['Overall'].describe(), decimals=2)

<b>Statische analyse column 'Age'</b>:

In [ ]:
np.round(fifa_source['Age'].describe(), decimals=0)

<b>Statische analyse column 'Release Clause'</b>:

In [ ]:
fifa_source['Release Clause'].describe()

<b>Histogram met Aantal spelers per 'Overall' waarde</b>:

In [ ]:
histogram = plt.hist(fifa_source['Overall'], edgecolor='black', linewidth=1.2)
plt.xlabel('Overall')
plt.ylabel('Amount')
histogram

<b>Histogram met Aantal spelers per 'Age' waarde</b>:

In [ ]:
histogram = plt.hist(fifa_source['Age'], edgecolor='black', linewidth=1.2)
plt.xlabel('Age')
plt.ylabel('Amount')
histogram

<b>Boxplot met column 'Overall'</b>:

In [ ]:
boxplot = plt.boxplot(fifa_source['Overall'], vert=False)
boxplot

<b>Scatter plot met column 'Overall' t.o.v. column 'Age'</b>:

In [ ]:
scatter = plt.scatter(fifa_source['Overall'], fifa_source['Age'])
plt.xlabel('Overall')
plt.ylabel('Age')
scatter

<b>Scatter plot met column 'Clean Wage' t.o.v. column 'Age'</b>:

In [ ]:
scatter = plt.scatter(fifa_source['Age'], fifa_source['Clean Wage'])
plt.xlabel('Age')
plt.ylabel('Wage')
scatter

## Externe dataset

Onze gekozen externe CSV-file dataset inladen: <br>
- fifa16.csv

De dataset bevat: <br>
1. Alle spelers uit FIFA 16.
2. Speelpositie en fysieke eigenschappen.
3. Persoonlijke spelersinformatie (bijv. nationaliteit, club, leeftijd, salaris etc.).

In [ ]:
extern_source = pd.read_csv('source/fifa16.csv')

De grootte van onze externe dataset:
- 7.18 MB
- Bevat 14881 rijen (dus 14880 spelers, want de eerste rij bevat alle kolommen)

In [ ]:
rows_extern_dataset = extern_source.shape[0] - 1
rows_extern_dataset

Een aantal rijen als voorbeeld voor hoe de externe dataset eruit ziet:

In [ ]:
extern_source.head(10)

<b>Verplichte onderzoeksvragen: </b>
   * *In hoeverre is de waarde van een speler te voorspellen aan de hand 
       van zijn belangrijkste kenmerken?*
   

<b>Concept-onderzoeksvragen: </b>
   - *In hoeverre is de positie van een spelers te voorspellen aan de hand van zijn kenmerken?*
   - *In hoeverre is de 'potential' van de destijds jonge talenten in fifa 16 (max 21 jaar oud met een goed potentie) uitgekomen in fifa 19?*

<b>Bonus (mocht er tijd over zijn)</b>: In hoeverre is het salaris van een speler te voorspellen aan de hand van zijn kenmerken? (Voorspelling: betere speler is hoger salaris, leeftijd speelt een rol, positie speelt een rol, nationaliteit speelt een rol)

# Uitwerking van de onderzoeksvragen
*Wij als team hebben ervoor gekozen om ieder onderzoeksvraag in een op zichzelf staand 'hoofdstuk' te behandelen, wij zijn er namelijk van overtuigd dat dit de notebook overzichtelijker maakt.*

### *1. "In hoeverre is de waarde van een speler te voorspellen aan de hand van zijn belangrijkste kenmerken?"*

Het is handig om eerst even de onderzoeksvraag te gaan splitsen:
- Wat weergeeft nou eigenlijk de <b>waarde</b> van een speler?
- Wat zijn de <b>"belangrijkste kenmerken"</b> van een speler?

Laten we hiervoor eerst even weer alle kolommen tonen van de FIFA 19 dataset:

In [ ]:
fifa_source.columns

Bij <b>waarde</b> zou je kunnen denken aan de kolommen:
- 'Clean Value'; de waarde van een speler in euro's.
- 'Clean Wage'; de salaris van de desbetreffende speler.
- 'Clean Release Clause'; een bedrag binnen het contract van een speler. Stel dat er wordt onderhandeld over een speler tussen 2 teams, en de team waar de speler zich momenteel bevind gaat <b>niet</b> akkoord, dan kan het team die interesse heeft de 'Release Clause' bedrag betalen. Als dit bedrag wordt betaald, <b>moet</b> het huidige team van die desbetreffende speler hem laten gaan. Dit bedrag zit over het algemeen <b>hoger</b> dan de werkelijke waarde van een speler.

In [ ]:
# ik gebruik de oorspronkelijke kolom met de juiste characters (bijv. 'M'), dit maakt het makkelijker om te lezen
fifa_source['Value']

In [ ]:
# ik gebruik de oorspronkelijke kolom met de juiste characters (bijv. 'M'), dit maakt het makkelijker om te lezen
fifa_source['Release Clause'] 

'Clean Release Clause' willen wij <b>niet</b> gaan gebruiken, aangezien dit niet de werkelijke waarde (in geld) van een speler weergeeft. Zoals je kunt zien is het bedrag hier namelijk een stuk hoger.

Dan blijven er dus 2 kolommen over;
- 'Clean Value'
- 'Clean Wage'

*Aangezien 'Clean Value' letterlijk een soort 'totale marktwaarde' van een speler weergeeft, gaan wij voor <b>'Clean Value'</b>.*

Bij <b>belangrijkste kenmerken</b> kan je denken aan vrijwel alle kenmerken van de speler, zoals:
- Weak Foot, Balance, ShotPower, Stamina, SprintSpeed, LM, CM
- etc...

Echter is er ook een kolom 'Overall':
- een getal (rating) van een speler. Dit geeft weer hoe goed een speler is t.o.v. de andere spelers binnen de dataset.<br>*(een soort 'gemiddelde' van de belangrijkste factoren/kenmerken van een speler)*

*Aangezien 'Overall' letterlijk een soort 'gemiddelde' is van alle kenmerken, gaan wij voor <b>'Overall'</b>.*

Er zijn spelers die als 'Clean Value' de waarde <b>0</b> hebben, deze spelers willen wij niet gaan gebruiken:

In [ ]:
value_filter = fifa_source['Clean Value'] != 0

fifa_source = fifa_source[value_filter]

Wij gaan namelijk de 'Clean Value' van de spelers iets aanpassen;
- in de scatterplot zien wij namelijk een <b>exponentiële groei</b>
- om het visualiseren wat duidelijker te maken, voeren we een <b>log</b> hierop uit, je wilt geen log op getal 0 uitvoeren
- bij het predicten van een nieuwe 'Clean Value' krijg je een log versie van dit getal terug, die gaan wij dan weer omzetten naar de behorende grootte om vervolgens zo te kunnen zien wat er gepredict is

In [ ]:
fifa_source['Clean Value Log'] = np.log(fifa_source['Clean Value'])

In [ ]:
scatter = hv.Scatter((fifa_source['Overall'], fifa_source['Clean Value Log']))
scatter.opts(title="'Overall' t.o.v. 'Value' (in euro's)", xlabel= 'Overall', ylabel='Value',
             bgcolor='lightgray', width=500, height=475)

scatter

Je ziet in deze *interactieve* Scatter Plot een duidelijk <b>positief correlatie</b> tussen de kolommen 'Value' en 'Overall'<br>(wat overigens ook wel te verwachten valt: *hoe hoger je overall, hoe meer je waard bent*).

- Hieruit concluderen wij dus een hoog voorspellingskracht.

Voor het voorspellen gaan wij dan ook gebruik maken van <b>Supervised Learning</b>;
- <b>Regression (lineare regressie)</b>, we gaan hier namelijk een *continue waarde* proberen te voorspellen

In [ ]:
linreg = LinearRegression()

Organiseren van de data: <b>features (eigenschappen)</b> en <b>target (resultaat)</b>:

In [ ]:
features = fifa_source[['Overall']]
target = fifa_source[['Clean Value Log']]

Het creëren van een <b>training- en validatieset</b>:

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target)

- 75% van de FIFA 19 dataset wordt gebruikt als <b>trainingset</b>.
- 25% van de FIFA 19 dataset wordt gebruikt als <b>testset</b>.

In [ ]:
trainingset = np.round(features_train.shape[0] / fifa_source.shape[0] * 100, decimals=0)
testset = np.round(features_test.shape[0] / fifa_source.shape[0] * 100, decimals=0)
print('Trainigset: {}% van de hele FIFA 19 dataset\nTestset: {}% van de hele FIFA 19 dataset'.format(trainingset, testset))

Het starten van 't <b>leerproces</b>:

In [ ]:
linreg.fit(features_train, target_train)

Een voorbeeld van een <b>predict</b>:

In [ ]:
messi_value_log = fifa_source['Clean Value Log'][0] # Messi is 94 Overall
predicted_value_log = linreg.predict([[94]]) ## Predicten met 94 Overall

messi_value = np.exp(messi_value_log)
predicted_value = np.exp(predicted_value_log[0][0])

print('Messi\'s value in log formaat: {}'.format("{:e}".format(messi_value_log)))
print('Predicted value in log formaat: {}\n'.format("{:e}".format(predicted_value_log[0][0])))

print('Messi\'s value: {}'.format("{:e}".format(messi_value)))
print('Predicted value: {}'.format("{:e}".format(predicted_value)))

Het model is nu getraint, wat we nu willen doen is het <b>valideren</b> van het model:
- hierbij gebruiken wij <b>Root Mean Squared Error</b> (RMSE)
- dit getal wil je zo laag mogelijk hebben t.o.v. de target, in dit geval dus <b>'Clean Value Log'</b>

In [ ]:
target_predict = linreg.predict(features_test)

In [ ]:
rmse = np.sqrt(metrics.mean_squared_error(target_test, target_predict))

rmse

Misschien kunnen we ook gebruik maken van een extra kolom om onze <b>RMSE</b> lager te krijgen;
- laten we de kolom <b>'Age'</b> proberen
- we volgen gewoon dezelfde stappen als hierboven, maar dan dus met een extra waarde

In [ ]:
features = fifa_source[['Overall', 'Age']]
target = fifa_source[['Clean Value Log']]

features_train, features_test, target_train, target_test = train_test_split(features, target)

linreg.fit(features_train, target_train)

target_predict = linreg.predict(features_test)

rmse = np.sqrt(metrics.mean_squared_error(target_test, target_predict))

rmse

Zoals je kunt zien is de RMSE met bijna de helft <b>verlaagd</b>!
- Hierbij wil ik het laten; als er namelijk meer kolommen worden toegevoegd, dan stijgt de RMSE (dit willen we natuurlijk niet)

<b>Conclusie</b>: 
- *'In hoeverre is de waarde van een speler te voorspellen aan de hand van zijn belangrijkste kenmerken?'*
- Het is natuurlijk lastig om te bepalen wat nou precies de *belangrijke* kenmerken van een speler zijn, maar doordat de kolom 'Overall' al een soort 'gemiddelde' van de belangrijke kenmerken is, is het gebruik van deze kolom een uitstekende keuze. Vooral in combinatie van de kolom 'Age', aangezien een leeftijd ook behoorlijk wat invloed heeft op de value van een speler. Als je als jonge speler een hoge Overall hebt, dan is het natuurlijk vanzelfsprekend dat hier een vrij hoge value aan gekoppeld gaat worden.
- Als je gaat kijken naar hoe laag de RMSE is t.o.v. de grootte van de 'log' versie van 'Value', dan is het een prima RMSE!

### *2. "In hoeverre is de positie van een spelers te voorspellen aan de hand van zijn kenmerken?"*

Om dit te kunnen voorspellen moet eerst het volgende bepaald worden:
- Aan de hand van welke gegevens gaan we de positie voorspellen?
- Welke posities zijn mogelijk in de dataset?

Voor de uitvoering van deze vraag gaat er een unsupervised machine learning techniek gebruikt worden. Namelijk de Gaussian Mixture Model. Dit betekend dat de dataset ook aangepast moet worden, deze bevat al per speler data over op welke positie hij normaliter speelt en ratings voor de verschillende posities in het spel. Dus maken wij een aparte dataset met de volgende gegevens:

'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes'

Deze gegevens zijn het meest kenmerkend voor de kwaliteiten van een speler en op welke positie hij het beste zou kunnen bespelen, zonder een directe link aan een positie te hebben.

In [ ]:
ds_vraag2 = pd.DataFrame(fifa_source[{'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes'}]
)
ds_vraag2

De mogelijke posities zijn in de dataset duidelijk weergegeven. Elke veldspeler heeft een waarde voor hoe goed ze functioneren op de volgende posities:
- LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB

Door een set spelers en hun waarden bij de posities te tonen valt meteen iets op, doelmannen hebben standaard een 0 voor elke veldpositie. Daarnaast ontbreekt in de dataset een kolom voor de positie van doelman, GK. Zoals in onderstaande tabel te zien.

In [ ]:
fifa_source.head(10)[{'Name', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB'}]

Aangezien veel van deze posities vrij sterk overeenkomen. Bijvoorbeeld LCM, CM en RCM zijn allemaal centrale middenvelders en vrijwel dezelfde positie. Voor dit onderzoek gaan we uit van vier groepen, Aanvallers, Middenvelders, Verdedigers en Doelmannen.

In [ ]:
gmm = GaussianMixture(n_components=4, covariance_type='full', n_init=5, random_state=0).fit(ds_vraag2)
gmm

Om de namen en posities van spelers mee te kunnen nemen in de uiteindelijke tabel nemen we die over uit de fifa_source document. Aangezien enkele spelers uit dit document gehaald zijn i.v.m. NaN waarden moeten de index nummers nog aangepast worden om weer te kloppen.Zoals onder te zien is was in de oude rij 538 leeg en in de nieuwe dataset is het gecorrigeerd.

In [ ]:
fifa_source_reset_index = fifa_source.reset_index()
fifa_source_reset_index[435:440]

Door het toevoegen van de Naam en Positie kolommen kan in onderstaand tabel makkelijk de uitslag van de gmm met de werkelijkheid vergeleken worden.

In [ ]:
test = gmm.predict_proba(ds_vraag2).round(3)
gmmresult = pd.DataFrame(test)
gmmresult['Naam'] = fifa_source_reset_index['Name']
gmmresult['Positie'] = fifa_source_reset_index['Position']
gmmresult['Aanvallers'] = gmmresult[0]
gmmresult['Middenvelders'] = gmmresult[2]
gmmresult['Verdedigers'] = gmmresult[3]
gmmresult['Doelmannen'] = gmmresult[1]
gmmresult = gmmresult.drop([0, 1, 2, 3], axis=1)
gmmresult[0:50]

En het resultaat voldoet grotendeels aan de verwachtingen, maar is soms opvallend. Veel spelers zijn duidelijk herkenbaar. De doelmannen zijn op het eerste oog 100% eruit gehaald. Veel Aanvallers zijn ook goed herkend. Enkele verdedigers zoals Alba (49), Thiago Silva (39) en Marcelo (35) zijn als middenvelders geïdentificeerd. Waar Alba en Marcelo vleugelverdedigers zijn die in het echt ook heel de flank bestrijken en bekend staan om hun aanvallende kwaliteiten, is Silva op het eerste oog een vreemde uitschieter. 

Een ander voorbeeld is P. Aubameyang, in het spel LM (linker middenvelder), door de gmm geïdentificeerd als Aanvaller, en in het echt is het ook een aanvaller die zowel in de Duitse Bundesliga als de Engelse Premier League meedeed en doet om de topscorerstitel. De resultaten van de gmm lijken in dit geval meer te stroken met de werkelijkheid. 

Uiteindelijk zijn dit ook de beperkingen van het kiezen van 4 componenten in de gmm. Er zijn aanvallende middenvelders, verdedigende middenvelders en middenvelders die een beetje van beiden doen. Centrumverdedigers zijn vooral verdedigend goed terwijl van vleugelverdedigers ook aanvallende bijdrages wordt verwacht. 

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive

kenmerk1 = widgets.Dropdown(
    options=["HeadingAccuracy", "ShortPassing", "Volleys", "Dribbling", "Curve", "FKAccuracy", "LongPassing", "BallControl", "Acceleration", "SprintSpeed", "Agility", "Reactions", "Balance", "ShotPower", "Jumping", "Stamina", "Strength", "LongShots", "Aggression", "Interceptions", "Positioning", "Vision", "Penalties", "Composure", "Marking", "StandingTackle", "SlidingTackle", "GKDiving", "GKHandling", "GKKicking", "GKPositioning", "GKReflexes"],
    value="ShortPassing",
    description='Kenmerk 1:',
    disabled=False,
)
kenmerk2 = widgets.Dropdown(
    options=["HeadingAccuracy", "ShortPassing", "Volleys", "Dribbling", "Curve", "FKAccuracy", "LongPassing", "BallControl", "Acceleration", "SprintSpeed", "Agility", "Reactions", "Balance", "ShotPower", "Jumping", "Stamina", "Strength", "LongShots", "Aggression", "Interceptions", "Positioning", "Vision", "Penalties", "Composure", "Marking", "StandingTackle", "SlidingTackle", "GKDiving", "GKHandling", "GKKicking", "GKPositioning", "GKReflexes"],
    value="Dribbling",
    description='Kenmerk 2:',
    disabled=False,
)

def plotit(kenmerk1, kenmerk2):
    size = 5 * test.max(1) ** 2
    labels = gmm.predict(ds_vraag2)
#     plt.figure(figsize=(12,6), dpi=80)
#     plt.xlabel(kenmerk1)
#     plt.ylabel(kenmerk2)
#     plt.scatter(ds_vraag2[kenmerk1], ds_vraag2[kenmerk2], c=labels, s=size, )
#     plt.title('Interactief scatterplot')
    x = ds_vraag2[kenmerk1]
    y = ds_vraag2[kenmerk2]
    c = labels
    s = size

    fig, ax = plt.subplots(figsize=(10,10))

    scatter = ax.scatter(x, y, c=c, s=s)
    classes = ['Aanvallers', 'Doelmannen', 'Middenvelders', 'Verdedigers']
    legend1 = ax.legend(handles=scatter.legend_elements()[0],
                    loc="upper left", bbox_to_anchor=(1,1), title="Posities", labels=classes)
    ax.set_xlabel(kenmerk1)
    ax.set_ylabel(kenmerk2)
    ax.add_artist(legend1)


In [ ]:
interactive(plotit, kenmerk1=kenmerk1, kenmerk2=kenmerk2)

In [ ]:
first_4000_gmmresult = gmmresult.loc[:10000]
aanvallers = first_4000_gmmresult['Positie'].isin(['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW'])
middenvelders = first_4000_gmmresult['Positie'].isin(['LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB'])
verdedigers = first_4000_gmmresult['Positie'].isin(['LB', 'LCB', 'CB', 'RCB', 'RB'])
doelmannen = first_4000_gmmresult['Positie'].isin(['GK'])
aan_mean = first_4000_gmmresult[aanvallers]['Aanvallers'].mean()
mid_mean = first_4000_gmmresult[middenvelders]['Middenvelders'].mean()
ver_mean = first_4000_gmmresult[verdedigers]['Verdedigers'].mean()
doel_mean = first_4000_gmmresult[doelmannen]['Doelmannen'].mean()
print("Aan de hand van de resultaten van de eerste 10000* voetballers gaan we berekenen of de gmm in de buurt komt van de werkelijkheid. Dit doen we in de volgende stappen.\n- Verdeel de mogelijke posities in het spel in groepen genaamd aanvallers, verdedigers, middenvelders en doelmannen.\n- Selecteer aan de hand van deze groepen de spelers die binnen deze groepen vallen. Hiermee creëer je vier datasets.\n- Bereken de gemiddelde waarde die geschat is per groep. Dus het gemiddelde van de kolom Aanvallers voor de groep aanvallers, enzovoort.\n- Hoe dichter bij de 1, hoe beter de voorspelling.\n")
print("Aanvallers = {}".format(aan_mean))
print("Middenvelders = {}".format(mid_mean))
print("Verdedigers = {}".format(ver_mean))
print("Doelmannen = {}\n".format(doel_mean))

mid_afwijkend_mean = first_4000_gmmresult[middenvelders]['Aanvallers'].mean()
ver_afwijkend_mean = first_4000_gmmresult[verdedigers]['Middenvelders'].mean()

print("Zoals eerder al opgemerkt, Aanvallers en Doelmannen worden erg goed voorspelt, Middenvelders en Verdedigers een stuk minder. Klopt het dat de middenvelders als aanvallers worden gezien en verdedigers als middenvelders?\n")
print("Middenvelders als aanvallers = {}".format(mid_afwijkend_mean))
print("Verdedigers als middenvelders = {}\n".format(ver_afwijkend_mean))

print("Als het totaal van deze waarden dicht bij 1 liggen is dit een juiste conclusie\n")
print("Totaal van normale en aanvallende middenvelders = {}".format(mid_afwijkend_mean+mid_mean))
print("Totaal van normale en middenvelder verdedigers = {}\n".format(ver_afwijkend_mean+ver_mean))



# Conclusie

- _In hoeverre is de positie van een spelers te voorspellen aan de hand van zijn kenmerken?_

De posities zijn goed te voorspellen aan de hand van de kenmerken van een speler. Vooral de aanvallers en doelmannen werden er goed uit gehaald. Dat is misschien ook wel logisch, dit zijn in principe de extremen op het veld. Doelmannen hebben een totaal andere set van vaardigheden en dit is in de dataset ook vertegenwoordigd met speciale kenmerken voor doelmannen. De meeste aanvallers zijn in het voetbal ook puur aanvallers en doen weinig verdedigende taken, het is te verwachten dat ze op aanvallende kenmerken hoog scoren en op verdedigende laag. 

Pas bij middenvelders en verdedigers komt er een uitdaging. Sommige middenvelders zijn zeer aanvallen ingesteld, zoals De Bruyne en Hazard, anderen zijn zeer verdedigend, bijvoorbeeld Kanté. Dit kan je ook terugzien in de resultaten, de aanvallend ingestelde spelers worden (deels) gezien als aanvallers. Bij verdedigers zie je dit vooral bij de backs. Niet verrassend, in veel teams wordt verwacht dat zij naast hun verdedigende taken ook aanvallend meedoen en opkomen over de vleugel. Het valt te verwachten dat deze spelers ook aanvallende kwaliteiten hebben.

Ook is er een verschil in de voorspelbaarheid tussen spelers op het hoogste niveau en lagere niveaus. Bij de spelers op laag niveau gaat het toch net iets moeilijker met de middenvelders en verdedigers. Waarschijnlijk omdat deze spelers in het algemeen al veel lagere waarden hebben voor hun kenmerken en de verschillen dus minder duidelijk zijn. Daarnaast zullen ze ook vaker niet goed zijn op een kenmerk waar je wel zou verwachten dat een speler op die positie goed in zou zijn. Vandaar dat ze ook op lager niveau spelen.

Een mogelijkheid om te resultaten nog verder te verbeter zou zijn om op meer clusters te zoeken. Aanvallers, Aanvallende middenvelders, Verdedigende Middenvelders, Vleugelverdedigers, Centrale verdedigers en doelmannen lijken op basis van de resultaten de categoriën om te verwachten en dus 6 clusters zou kunnen werken.

### *3. "In hoeverre is de 'potential' van de destijds jonge talenten in fifa 16 (max 21 jaar oud met een goed potentie) uitgekomen in fifa 19?"*

Voor het beantwoorden van deze onderzoeksvraag gaan wij gebruik maken van voornamelijk 4 kolommen:
- 'Age'
- 'Potential' (FIFA16)
- 'Overall' (FIFA16)
- 'Overall' (FIFA19)

Om de 'Potential' en 'Overall' van de twee verschillende datasets met elkaar te kunnen vergelijken, moeten we een aantal kleine aanpassingen gaan maken in onze FIFA19 dataset;
- de kolomnamen worden aangepast (dit wordt gedaan zodat we die kunnen mergen in een nieuwe dataset met alle spelers onder de 21 jaar)
- vervolgens mergen we de twee datasets op id (FIFA16 = <b>'sofifa_id'</b>, FIFA19 = <b>'ID'</b>)

In [ ]:
fifa_source = fifa_source.rename(columns={'Age': "age19", "Overall": "overall19", "Potential": "potential19"})

df = pd.merge(extern_source, fifa_source[['ID', 'age19', 'overall19', 'potential19']], how='left', left_on='sofifa_id', right_on='ID')

Wat we eerst willen doen is;
- de externe FIFA16 dataset filteren, zodanig dat alle jonge talenten <b>(onder de 21 jaar)</b> over blijven

In [ ]:
df = df.loc[df['age'] <= 21]

Het is natuurlijk logisch dat er speler zijn die wel in de FIFA16 dataset zaten, maar niet meer in de FIFA19 dataset.
- we checken op de kolom <b>'age19'</b>, dit zou de age van deze speler moeten zijn in de FIFA19 dataset
- als deze kolom in de rij van de desbetreffende speler <b>geen</b> waarde bevat, dan is de speler dus uit het spel gehaald

In [ ]:
total_players = df.shape[0]
removed_players = pd.isnull(df['age19']).sum().sum()

print("Er zijn in totaal {} spelers, hiervan zijn er {} spelers die uit het spel zijn gehaald.".format(total_players, removed_players))

Deze 'verdwenen' spelers willen wij niet gaan gebruiken bij het beantwoorden van de onderzoeksvraag, aangezien de spelers n.v.t. zijn:

In [ ]:
df = df[df['age19'].notna()]

print("Resterende spelers na verwijderen van 'verdewenen' spelers: {}".format(df.shape[0]))

Nu moeten wij onszelf de volgende vraag stellen;
- wat is nou een <b>goede potentie?</b>

Voor een goede potentie kiezen wij de 'Overall' van de <b>top 5% van alle spelers uit de FIFA16 dataset</b>. Hiervoor gaan we gebruik maken van de standaarddeviatie:

In [ ]:
mu = extern_source['overall'].mean()
sigma = extern_source['overall'].std()

top5 = mu + 1.64*sigma

print("De standaarddeviatie (afgerond): {}".format(np.round(top5)))

Het blijkt dat de top 5% beste spelers uit FIFA16 <b>boven de 77.29</b> 'Overall' score zit. Deze waarde gaan wij dan ook als datapunt gebruiken:
- wij willen zien of de spelers boven dit punt uit zijn gekomen

In [ ]:
overallverdeling = stats.norm(loc=mu,scale=sigma)

x = np.linspace(mu-3*sigma, mu+3*sigma, 100)
y = overallverdeling.pdf(x)

plt.figure(figsize=(12,6), dpi=80)

plt.plot(x,y)

plt.xticks(np.linspace(mu-3*sigma,mu+3*sigma,7))
plt.xlabel("Overall")
plt.axvline(top5, c='r', label="top 5 %")

plt.legend()

plt.show()

Daarnet hebben wij alle spelers uit de FIFA16 dataset *onder* de 21 jaar gefilterd uit de dataset. Wat we nu gaan doen is uit al deze 21-jarige spelers alleen de spelers die *boven* de 77.29 zijn gekomen eruit filteren.

In [ ]:
df = df.loc[df['potential'] >= top5]

In [ ]:
plt.scatter(df['age'], df['potential'], label='Potential 16', alpha=.1)
plt.scatter(df['age'], df['overall'], label='Overall 16', alpha=.1)
plt.scatter(df['age19'], df['overall19'], label='Overall 19', alpha=.1)

lgt = plt.legend(loc='best', fontsize=16)
for lh in lgt.legendHandles:
    lh.set_alpha(1)
plt.xlabel('Age')
plt.ylabel('Overall')

Er zijn 40 spelers waarvan de 'Overall' in FIFA19 <b>groter of gelijk (>=)</b> is dan de 'Potential' die ze hadden in FIFA16:
- Frenkie de Jong zit ook in 't lijstje, wat natuurlijk logisch is (van Willem II naar FC Barcelona)

In [ ]:
df.loc[df['overall19'] >= df['potential']]

<b>Conclusie</b>: 
- *'In hoeverre is de 'potential' van de destijds jonge talenten in fifa 16 (max 21 jaar oud met een goed potentie) uitgekomen in fifa 19?'*
- Je ziet in de scatterplot dat de 'Overall' van deze talenten in FIFA19 een heel stuk lager zitten t.o.v. de 'Potential' die ze hadden in FIFA16.
- Onze conclusie is dat van de getalenteerde spelers uit FIFA16 de meeste nog niet hun potentie bereikt hebben.
- De potentie had echter ook al bereikt kunnen zijn, maar aangezien er 3 jaar tussen de twee verschillende spellen zitten, kan de potentie weer gedaald zijn.